# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [24]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine

import nltk
nltk.download('punkt')

from nltk import word_tokenize

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.multioutput import MultiOutputClassifier
from sklearn.naive_bayes import MultinomialNB

from sklearn.metrics import classification_report

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///../dataset/DisasterResponse.db')
df = pd.read_sql_table('disaster_response', engine)
print(df.shape)
df.head()

(26216, 40)


,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
X = df['message']
y = df.drop(['id', 'message', 'original', 'genre'], axis=1)

In [4]:
y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    tokens = word_tokenize(text)
    
    return tokens

In [6]:
### Testing the function
tokenize('A cat pulls a dog\'s tail.')

['A', 'cat', 'pulls', 'a', 'dog', "'s", 'tail', '.']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
pipeline = Pipeline([
        ('tfidf', TfidfVectorizer()),
        ('clf', MultiOutputClassifier(MultinomialNB()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=50)

In [9]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf', MultiOutputClassifier(estimator=MultinomialNB()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
def column_metrics(y_true, y_pred, column):
    """Return metrics for the column"""
    report = classification_report(y_true[column], y_pred[column], zero_division=0)
    report_dict = classification_report(y_true[column], y_pred[column], zero_division=0, output_dict=True)
    
    return report, report_dict

In [16]:
def model_metrics(column_metrics):
    """Return metrics for the ML model"""

    accuracy = 0
    precision = 0
    recall = 0
    f1_score = 0
    for metrics in column_metrics:
        accuracy += metrics['accuracy']
        precision += metrics['macro avg']['precision']
        recall += metrics['macro avg']['recall']
        f1_score += metrics['macro avg']['f1-score']

    metrics_size = len(column_metrics)

    accuracy /= metrics_size
    precision /= metrics_size
    recall /= metrics_size
    f1_score /= metrics_size
    
    return accuracy, precision, recall, f1_score

In [12]:
y_pred = pipeline.predict(X_test)
y_pred_df = pd.DataFrame(y_pred, columns=y_test.columns)

metrics = []
for column in y_test.columns:
    report, report_dict = column_metrics(y_test, y_pred_df, column)
    metrics.append(report_dict)
    
    print(column)
    print(report)

related
              precision    recall  f1-score   support

           0       0.86      0.06      0.12      1517
           1       0.78      1.00      0.87      5037

    accuracy                           0.78      6554
   macro avg       0.82      0.53      0.50      6554
weighted avg       0.80      0.78      0.70      6554

request
              precision    recall  f1-score   support

           0       0.86      0.99      0.92      5433
           1       0.89      0.20      0.33      1121

    accuracy                           0.86      6554
   macro avg       0.87      0.60      0.63      6554
weighted avg       0.86      0.86      0.82      6554

offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6511
           1       0.00      0.00      0.00        43

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      0.99      0.99      655

In [13]:
def print_model_metrics(accuracy, precision, recall, f1_score):
    """Print the model metrics in a prettier way"""
    
    print('The model metrics are:')
    print('Accuracy: {:.2f}%'.format(accuracy))
    print('Precision: {:.2f}%'.format(precision))
    print('Recall: {:.2f}%'.format(recall))
    print('F1 score: {:.2f}%'.format(f1_score))

In [22]:
accuracy, precision, recall, f1_score = model_metrics(metrics)

print_model_metrics(accuracy, precision, recall, f1_score)

The model metrics are:
Accuracy: 0.94%
Precision: 0.60%
Recall: 0.53%
F1 score: 0.52%


### 6. Improve your model
Use grid search to find better parameters. 

In [17]:
pipeline.get_params()

{'memory': None,
 'steps': [('tfidf', TfidfVectorizer()),
  ('clf', MultiOutputClassifier(estimator=MultinomialNB()))],
 'verbose': False,
 'tfidf': TfidfVectorizer(),
 'clf': MultiOutputClassifier(estimator=MultinomialNB()),
 'tfidf__analyzer': 'word',
 'tfidf__binary': False,
 'tfidf__decode_error': 'strict',
 'tfidf__dtype': numpy.float64,
 'tfidf__encoding': 'utf-8',
 'tfidf__input': 'content',
 'tfidf__lowercase': True,
 'tfidf__max_df': 1.0,
 'tfidf__max_features': None,
 'tfidf__min_df': 1,
 'tfidf__ngram_range': (1, 1),
 'tfidf__norm': 'l2',
 'tfidf__preprocessor': None,
 'tfidf__smooth_idf': True,
 'tfidf__stop_words': None,
 'tfidf__strip_accents': None,
 'tfidf__sublinear_tf': False,
 'tfidf__token_pattern': '(?u)\\b\\w\\w+\\b',
 'tfidf__tokenizer': None,
 'tfidf__use_idf': True,
 'tfidf__vocabulary': None,
 'clf__estimator__alpha': 1.0,
 'clf__estimator__class_prior': None,
 'clf__estimator__fit_prior': True,
 'clf__estimator': MultinomialNB(),
 'clf__n_jobs': None}

In [19]:
parameters = {
    'tfidf__ngram_range': ((1, 1), (2, 2)),
    'clf__estimator__alpha': (0.5, 1.0)
}

cv = GridSearchCV(pipeline, parameters)

In [20]:
cv.fit(X, y)

GridSearchCV(estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=MultinomialNB()))]),
             param_grid={'clf__estimator__alpha': (0.5, 1.0),
                         'tfidf__ngram_range': ((1, 1), (2, 2))})

In [27]:
cv.best_params_

{'clf__estimator__alpha': 1.0, 'tfidf__ngram_range': (2, 2)}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [23]:
y_pred = cv.predict(X_test)
y_pred_df = pd.DataFrame(y_pred, columns=y_test.columns)

metrics = []
for column in y_test.columns:
    report, report_dict = column_metrics(y_test, y_pred_df, column)
    metrics.append(report_dict)

accuracy, precision, recall, f1_score = model_metrics(metrics)

print_model_metrics(accuracy, precision, recall, f1_score)

The model metrics are:
Accuracy: 0.95%
Precision: 0.64%
Recall: 0.55%
F1 score: 0.55%


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [25]:
pipeline = Pipeline([
        ('tfidf', TfidfVectorizer()),
        ('clf', MultiOutputClassifier(SVC()))
    ])

In [26]:
pipeline.get_params()

{'memory': None,
 'steps': [('tfidf', TfidfVectorizer()),
  ('clf', MultiOutputClassifier(estimator=SVC()))],
 'verbose': False,
 'tfidf': TfidfVectorizer(),
 'clf': MultiOutputClassifier(estimator=SVC()),
 'tfidf__analyzer': 'word',
 'tfidf__binary': False,
 'tfidf__decode_error': 'strict',
 'tfidf__dtype': numpy.float64,
 'tfidf__encoding': 'utf-8',
 'tfidf__input': 'content',
 'tfidf__lowercase': True,
 'tfidf__max_df': 1.0,
 'tfidf__max_features': None,
 'tfidf__min_df': 1,
 'tfidf__ngram_range': (1, 1),
 'tfidf__norm': 'l2',
 'tfidf__preprocessor': None,
 'tfidf__smooth_idf': True,
 'tfidf__stop_words': None,
 'tfidf__strip_accents': None,
 'tfidf__sublinear_tf': False,
 'tfidf__token_pattern': '(?u)\\b\\w\\w+\\b',
 'tfidf__tokenizer': None,
 'tfidf__use_idf': True,
 'tfidf__vocabulary': None,
 'clf__estimator__C': 1.0,
 'clf__estimator__break_ties': False,
 'clf__estimator__cache_size': 200,
 'clf__estimator__class_weight': None,
 'clf__estimator__coef0': 0.0,
 'clf__estimator__d

In [28]:
parameters = {
    'tfidf__ngram_range': ((1, 1), (2, 2), (3, 3)),
    'tfidf__min_df': (1, 2, 3),
    'tfidf__max_features': (100, 1000, 10000, None),
    'clf__estimator__kernel': ('rbf', 'sigmoid'),
    'clf__estimator__C': (0.5, 1.0, 2.5),
    'clf__estimator__gamma': ('scale', 'auto'),
    'clf__estimator__class_weight': (None, 'balanced')
}

cv = GridSearchCV(pipeline, parameters)

In [29]:
cv.fit(X, y)

/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/conda/lib/python3.9/site-packages/sklearn/pipeline.py", line 346, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/opt/conda/lib/python3.9/site-packages/sklearn/multioutput.py", line 368, in fit
    super().fit(X, Y, sample_weight, **fit_params)
  File "/opt/conda/lib/python3.9/site-packages/sklearn/multioutput.py", line 178, in fit
    self.estimators_ = Parallel(n_jobs=self.n_jobs)(
  File "/opt/conda/lib/python3.9/site-packages/joblib/parallel.py", line 1044, in __call__
    while self.dispatch_one_batch(iterator):
  File "/opt/conda/li

KeyboardInterrupt: 

In [ ]:
cv.best_params_

In [ ]:
y_pred = cv.predict(X_test)
y_pred_df = pd.DataFrame(y_pred, columns=y_test.columns)

metrics = []
for column in y_test.columns:
    report, report_dict = column_metrics(y_test, y_pred_df, column)
    metrics.append(report_dict)

accuracy, precision, recall, f1_score = model_metrics(metrics)

print_model_metrics(accuracy, precision, recall, f1_score)

### 9. Export your model as a pickle file

In [ ]:
import pickle

filename = 'best_model.pkl'
pickle.dump(cv.best_estimator_, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.